In [1]:
import os
import sys

import matplotlib.pyplot as plt
from hydra import compose, initialize

MODULES_PATH = os.path.dirname(os.path.dirname(os.getcwd()))

if MODULES_PATH not in sys.path:
    sys.path.append(MODULES_PATH)
    
import math
import torch
import torch.nn as nn

import pytorch_lightning as lightning
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint 

import dev.utils as utils

from dev.ssm_modules.dynamics import DenseGaussianDynamics
from dev.ssm_modules.likelihoods import GaussianLikelihood
from dev.smoothers.lightning_trainers import LightningNonlinearSSM
from dev.ssm_modules.dynamics import DenseGaussianInitialCondition
from dev.ssm_modules.encoders import LocalEncoderLRMvn, BackwardEncoderLRMvn
from dev.smoothers.nonlinear_smoother_causal import NonlinearFilter, LowRankNonlinearStateSpaceModel

In [3]:
torch.cuda.empty_cache()

In [5]:
"""config"""

initialize(version_base=None, config_path="", job_name="lds")
cfg = compose(config_name="config")
lightning.seed_everything(cfg.seed, workers=True)
torch.set_default_dtype(torch.float32)

Seed set to 1234


In [7]:
"""generate data -- 2d oscillator with decay"""

n_trials = 500
n_neurons = 100
n_time_bins = 250

C = utils.FanInLinear(cfg.n_latents, n_neurons, device=cfg.device).requires_grad_(False)
Q_diag = 1e-3 * torch.ones(cfg.n_latents, device=cfg.device)
Q_0_diag = 2. * torch.ones(cfg.n_latents, device=cfg.device)
R_diag = 1e-1 * torch.ones(n_neurons, device=cfg.device)
m_0 = torch.zeros(cfg.n_latents, device=cfg.device)

mean_fn = utils.VdpDynamicsModel()

z = utils.sample_gauss_z(mean_fn, Q_diag, m_0, Q_0_diag, n_trials, n_time_bins)
y = C(z) + torch.sqrt(R_diag) * torch.randn((n_trials, n_time_bins, n_neurons), device=cfg.device)
y = y.detach()

y_train, z_train = y[:2*n_trials//3], z[:2*n_trials//3]
y_valid, z_valid = y[2*n_trials//3:], z[2*n_trials//3:]

In [9]:
y_train_dataset = torch.utils.data.TensorDataset(y_train,)
y_valid_dataset = torch.utils.data.TensorDataset(y_valid,)

train_dataloader = torch.utils.data.DataLoader(
    y_train_dataset,
    batch_size=cfg.batch_sz,
    shuffle=True,
    num_workers=9
)

valid_dataloader = torch.utils.data.DataLoader(
    y_valid_dataset,
    batch_size=cfg.batch_sz,
    shuffle=True,
)

In [11]:
"""likelihood pdf"""

likelihood_pdf = GaussianLikelihood(C, n_neurons, R_diag, device=cfg.device)

In [13]:
"""dynamics module"""

dynamics_fn = utils.build_gru_dynamics_function(cfg.n_latents, cfg.n_hidden_dynamics, device=cfg.device)
dynamics_mod = DenseGaussianDynamics(dynamics_fn, cfg.n_latents, Q_diag, device=cfg.device)

In [15]:
"""initial condition"""

initial_condition_pdf = DenseGaussianInitialCondition(cfg.n_latents, m_0, Q_0_diag, device=cfg.device)

In [17]:
"""local/backward encoder"""

backward_encoder = BackwardEncoderLRMvn(cfg.n_latents, cfg.n_hidden_backward, cfg.n_latents,
                                        rank_local=cfg.rank_local, rank_backward=cfg.rank_backward,
                                        device=cfg.device)

local_encoder = LocalEncoderLRMvn(cfg.n_latents, n_neurons, cfg.n_hidden_local, cfg.n_latents, rank=cfg.rank_local,
                                  device=cfg.device, dropout=cfg.p_local_dropout)

nl_filter = NonlinearFilter(dynamics_mod, initial_condition_pdf, device=cfg.device)

In [19]:
"""sequence vae"""

ssm = LowRankNonlinearStateSpaceModel(dynamics_mod, likelihood_pdf, initial_condition_pdf, backward_encoder,
                                      local_encoder, nl_filter, device=cfg.device)

In [21]:
"""lightning"""

seq_vae = LightningNonlinearSSM(ssm, cfg)

csv_logger = CSVLogger('logs/', name=f'r_y_{cfg.rank_local}_r_b_{cfg.rank_backward}', version='noncausal')
ckpt_callback = ModelCheckpoint(save_top_k=3, monitor='valid_loss', mode='min', dirpath='ckpts/',
                                filename='{epoch:0}_{valid_loss}')

trainer = lightning.Trainer(max_epochs=cfg.n_epochs,
                            gradient_clip_val=1.0,
                            default_root_dir='lightning/',
                            callbacks=[ckpt_callback],
                            logger=csv_logger
                            )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model=seq_vae, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)
torch.save(ckpt_callback.best_model_path, 'ckpts/best_model_path.pt')

/opt/anaconda3/envs/xfads/lib/python3.9/site-packages/lightning_fabric/loggers/csv_logs.py:268: Experiment logs directory logs/r_y_2_r_b_2/noncausal exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!

  | Name | Type                            | Params
---------------------------------------------------------
0 | ssm  | LowRankNonlinearStateSpaceModel | 41.3 K
---------------------------------------------------------
41.0 K    Trainable params
300       Non-trainable params
41.3 K    Total params
0.165     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/anaconda3/envs/xfads/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/opt/anaconda3/envs/xfads/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


/opt/anaconda3/envs/xfads/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
/opt/anaconda3/envs/xfads/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 3/3 [01:52<00:00,  0.03it/s, v_num=usal]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 3/3 [01:31<00:00,  0.03it/s, v_num=usal]    
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 3/3 [01:42<00:00,  0.03it/s, v_num=usal]    
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 3/3 [01:28<00:00,  0.03it/s, v_num=usal]    
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 3/3 [01:39<00:00,  0.03it/s, v_num=usal]    
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 3/3 [01:44<00:00,  0.03it/s, v_num=usal]    
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 6: 100%|██████████| 3/3 [01:36<00:00,  0.03it/s, v_num=usal]    
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 7: 100%|██████████| 3/3 [01:24<00:00,  0.04it/s, v_num=usal]    
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 8: 100%|██████████| 3/3 [01:34<00:00,  0.03it/s, v_num=usal]  